In [54]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tpot import TPOTClassifier
import hyperopt as hp
from hyperopt import hp, tpe, fmin, STATUS_OK, Trials
#Reading all the necessary files
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print(train_df.shape)
#Dropping cabin column
train_df = train_df.drop(columns = ["Name","Ticket", "Cabin"], axis = 1)
test_df = test_df.drop(columns = ["Name", "Ticket", "Cabin"], axis = 1)
#Handling missing values
train_df["Age"].fillna(train_df['Age'].mean(), inplace = True)
train_df["Embarked"].fillna(train_df['Embarked'].mode()[0], inplace = True)
test_df["Age"].fillna(test_df['Age'].mean(), inplace = True)
test_df["Fare"].fillna(test_df['Fare'].mean(), inplace = True)


train_dummies = pd.get_dummies(train_df, drop_first = True)
test_dummies = pd.get_dummies(test_df, drop_first = True)

X = train_dummies.drop("Survived", axis = 1)
y = train_dummies["Survived"]
Z = test_dummies

# Set up space dictionary with specified hyperparameters
space = {'max_depth': hp.quniform('max_depth', 2, 11, 1),'learning_rate': hp.uniform('learning_rate', 0.001,0.9),
         'subsample': hp.uniform('subsample', 0.1,0.9),'colsample_bytree': hp.uniform('colsample_bytree',0.1,0.9)}





#Submission = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': Predictions})
#Submission.to_csv("Submission.csv", index = False)
#print("Mission Successful")

(891, 12)


In [55]:
# Set up objective function
def objective(params):
	  params = {'max_depth': int(params['max_depth']),'learning_rate': params['learning_rate'],
             'subsample': params['subsample'], 'colsample_bytree': params['colsample_bytree']}
	  model = xgb.XGBClassifier(n_estimators=500, **params)
	  best_score = cross_val_score(model, X, y, scoring='accuracy', cv=10, n_jobs=4).mean()
	  loss = 1 - best_score
	  return loss





In [56]:
# Run the algorithm
best_results = fmin(fn=objective,space=space, max_evals=20, rstate=np.random.default_rng(42), algo=tpe.suggest)
print(best_results)

100%|██████████| 20/20 [00:33<00:00,  1.70s/trial, best loss: 0.18968789013732823]
{'colsample_bytree': 0.8791583573171619, 'learning_rate': 0.0163699242630223, 'max_depth': 10.0, 'subsample': 0.2859685099756776}


In [57]:
best_params = {
               "max_depth": int(best_results["max_depth"]),
               "subsample": best_results["subsample"],
               "colsample_bytree": best_results["colsample_bytree"]}
model = xgb.XGBClassifier(n_estimators = 300, **best_params)
model.fit(X, y)

# Making predictions on the test dataset
predictions = model.predict(Z)

In [ ]:
!pip install hyperopt

In [58]:
Submission = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': Predictions})
Submission.to_csv("Submission.csv", index = False)
print("Mission Successful")

Mission Successful
